In [1]:
import torch
import torch.optim as optim
import torch.utils.data as Data
import torch.nn as nn
import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing

ModuleNotFoundError: No module named 'EEG_cnn'

### Loading 

### translating signals to images

In [1]:
from cycle_gan.models import Discriminator
from cycle_gan.models import Generator
from cycle_gan.utils import ReplayBuffer
from cycle_gan.utils import LambdaLR
from cycle_gan.utils import Logger
from cycle_gan.utils import weights_init_normal
import torch

In [13]:
netG_A2B = Generator(1, 1)
netG_B2A = Generator(1,1)
netD_A = Discriminator(1)
netD_B = Discriminator(1)

netG_A2B.cuda()
netG_B2A.cuda()
netD_A.cuda()
netD_B.cuda()

Discriminator(
  (model): Sequential(
    (0): Conv2d(1, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (1): LeakyReLU(negative_slope=0.2, inplace=True)
    (2): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (3): InstanceNorm2d(128, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (4): LeakyReLU(negative_slope=0.2, inplace=True)
    (5): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (6): InstanceNorm2d(256, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (7): LeakyReLU(negative_slope=0.2, inplace=True)
    (8): Conv2d(256, 512, kernel_size=(4, 4), stride=(1, 1), padding=(1, 1))
    (9): InstanceNorm2d(512, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (10): LeakyReLU(negative_slope=0.2, inplace=True)
    (11): Conv2d(512, 1, kernel_size=(4, 4), stride=(1, 1), padding=(1, 1))
  )
)

In [14]:
netG_A2B.apply(weights_init_normal)
netG_B2A.apply(weights_init_normal)
netD_A.apply(weights_init_normal)
netD_B.apply(weights_init_normal)


Discriminator(
  (model): Sequential(
    (0): Conv2d(1, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (1): LeakyReLU(negative_slope=0.2, inplace=True)
    (2): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (3): InstanceNorm2d(128, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (4): LeakyReLU(negative_slope=0.2, inplace=True)
    (5): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (6): InstanceNorm2d(256, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (7): LeakyReLU(negative_slope=0.2, inplace=True)
    (8): Conv2d(256, 512, kernel_size=(4, 4), stride=(1, 1), padding=(1, 1))
    (9): InstanceNorm2d(512, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (10): LeakyReLU(negative_slope=0.2, inplace=True)
    (11): Conv2d(512, 1, kernel_size=(4, 4), stride=(1, 1), padding=(1, 1))
  )
)

In [15]:
criterion_GAN = torch.nn.MSELoss()
criterion_cycle = torch.nn.L1Loss()
criterion_identity = torch.nn.L1Loss()

In [16]:
import itertools
optimizer_G = torch.optim.Adam(itertools.chain(netG_A2B.parameters(), netG_B2A.parameters()),
                                lr=0.0001, betas=(0.5, 0.999))
optimizer_D_A = torch.optim.Adam(netD_A.parameters(), lr=0.0002, betas=(0.5, 0.999))
optimizer_D_B = torch.optim.Adam(netD_B.parameters(), lr=0.0002, betas=(0.5, 0.999))

lr_scheduler_G = torch.optim.lr_scheduler.LambdaLR(optimizer_G, lr_lambda=LambdaLR(200, 0, 100).step)
lr_scheduler_D_A = torch.optim.lr_scheduler.LambdaLR(optimizer_D_A, lr_lambda=LambdaLR(200, 0, 100).step)
lr_scheduler_D_B = torch.optim.lr_scheduler.LambdaLR(optimizer_D_B, lr_lambda=LambdaLR(200, 0, 100).step)

In [6]:
#run this cell everytime that we need to continue the training from a previous run
# loading the models
netD_A.load_state_dict(torch.load('output_EMG/netD_A.pth'))
netD_B.load_state_dict(torch.load('output_EMG/netD_B.pth'))
netG_A2B.load_state_dict(torch.load('output_EMG/netG_A2B.pth'))
netG_B2A.load_state_dict(torch.load('output_EMG/netG_B2A.pth'))

# loading optimizers
optimizer_D_A.load_state_dict(torch.load('output_EMG/optimizer_D_A.pth'))
optimizer_D_B.load_state_dict(torch.load('output_EMG/optimizer_D_B.pth'))
optimizer_G.load_state_dict(torch.load('output_EMG/optimizer_G.pth'))

# loading schedulers
lr_scheduler_D_A.load_state_dict(torch.load('output_EMG/lr_scheduler_D_A.pth'))
lr_scheduler_D_B.load_state_dict(torch.load('output_EMG/lr_scheduler_D_B.pth'))
lr_scheduler_G.load_state_dict(torch.load('output_EMG/lr_scheduler_G.pth'))

# Mover os modelos para a GPU, se disponível
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
netD_A.to(device)
netD_B.to(device)
netG_A2B.to(device)
netG_B2A.to(device)

# adapting the optimizers to rhe right device
for state in optimizer_D_A.state.values():
    for k, v in state.items():
        if isinstance(v, torch.Tensor):
            state[k] = v.to(device)

for state in optimizer_D_B.state.values():
    for k, v in state.items():
        if isinstance(v, torch.Tensor):
            state[k] = v.to(device)

for state in optimizer_G.state.values():
    for k, v in state.items():
        if isinstance(v, torch.Tensor):
            state[k] = v.to(device)

In [17]:
batch_size = 5
from torch.autograd import Variable
from torch.utils.data import TensorDataset, DataLoader


Tensor = torch.cuda.FloatTensor
input_A = Tensor(batch_size, 1, 256, 256)
input_B = Tensor(batch_size, 1, 256, 256)
target_real = Variable(Tensor(batch_size).fill_(1.0), requires_grad=False)
target_fake = Variable(Tensor(batch_size).fill_(0.0), requires_grad=False)

fake_A_buffer = ReplayBuffer()
fake_B_buffer = ReplayBuffer()

In [18]:
class ConvertToSingleChannel(object):
    def __call__(self, tensor):
        if tensor.dim() != 1  and tensor.size(0) > 1:
            #ignoring the the last dimension from what we saw yesterday
            tensor = torch.mean(tensor[:3, :, : ], dim=0, keepdim=True)
            #tensor = tensor[0, :, : ].unsqueeze(0)
        return tensor

In [19]:
from PIL import Image
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from cycle_gan.datasets import ImageDataset

transforms_ = [ transforms.Resize(int(256*1.12), Image.BICUBIC),
                transforms.RandomCrop(256), 
                transforms.ToTensor(),
                ConvertToSingleChannel(),
                transforms.Normalize((0.5), (0.5))
               ]
dataloader = DataLoader(ImageDataset('EMG_cyclegan_dataset/', transforms_ = transforms_, unaligned=True), 
                        batch_size=batch_size, shuffle=True, num_workers=4)


In [20]:
import gc
torch.cuda.empty_cache()
gc.collect()

496

In [21]:
epochs = 125
logger = Logger(epochs, len(dataloader))

Setting up a new session...


In [ ]:
for epoch in range(1, epochs + 1):
    for i, batch in enumerate(dataloader):
        # Set model input
        real_A = Variable(input_A.copy_(batch['A']))
        real_B = Variable(input_B.copy_(batch['B']))

        ###### Generators A2B and B2A ######
        optimizer_G.zero_grad()

        # Identity loss
        # G_A2B(B) should equal B if real B is fed
        same_B = netG_A2B(real_B)
        loss_identity_B = criterion_identity(same_B, real_B)*5.0
        # G_B2A(A) should equal A if real A is fed
        same_A = netG_B2A(real_A)
        loss_identity_A = criterion_identity(same_A, real_A)*5.0

        # GAN loss
        fake_B = netG_A2B(real_A)
        pred_fake = netD_B(fake_B)
        loss_GAN_A2B = criterion_GAN(pred_fake, target_real)

        fake_A = netG_B2A(real_B)
        pred_fake = netD_A(fake_A)
        loss_GAN_B2A = criterion_GAN(pred_fake, target_real)

        # Cycle loss
        recovered_A = netG_B2A(fake_B)
        loss_cycle_ABA = criterion_cycle(recovered_A, real_A)*10.0

        recovered_B = netG_A2B(fake_A)
        loss_cycle_BAB = criterion_cycle(recovered_B, real_B)*10.0

        # Total loss
        loss_G = loss_identity_A + loss_identity_B + loss_GAN_A2B + loss_GAN_B2A + loss_cycle_ABA + loss_cycle_BAB
        loss_G.backward()
        
        optimizer_G.step()
        ###################################

        ###### Discriminator A ######
        optimizer_D_A.zero_grad()

        # Real loss
        pred_real = netD_A(real_A)
        loss_D_real = criterion_GAN(pred_real, target_real)

        # Fake loss
        fake_A = fake_A_buffer.push_and_pop(fake_A)
        pred_fake = netD_A(fake_A.detach())
        loss_D_fake = criterion_GAN(pred_fake, target_fake)

        # Total loss
        loss_D_A = (loss_D_real + loss_D_fake)*0.5
        loss_D_A.backward()

        optimizer_D_A.step()
        ###################################

        ###### Discriminator B ######
        optimizer_D_B.zero_grad()

        # Real loss
        pred_real = netD_B(real_B)
        loss_D_real = criterion_GAN(pred_real, target_real)
        
        # Fake loss
        fake_B = fake_B_buffer.push_and_pop(fake_B)
        pred_fake = netD_B(fake_B.detach())
        loss_D_fake = criterion_GAN(pred_fake, target_fake)

        # Total loss
        loss_D_B = (loss_D_real + loss_D_fake)*0.5
        loss_D_B.backward()

        optimizer_D_B.step()
        ###################################

        # Progress report (http://localhost:8097)
        logger.log({'loss_G': loss_G, 'loss_G_identity': (loss_identity_A + loss_identity_B), 'loss_G_GAN': (loss_GAN_A2B + loss_GAN_B2A),
                    'loss_G_cycle': (loss_cycle_ABA + loss_cycle_BAB), 'loss_D': (loss_D_A + loss_D_B)}, 
                    images={'real_A': real_A, 'real_B': real_B, 'fake_A': fake_A, 'fake_B': fake_B})
    
    # Update learning rates
    lr_scheduler_G.step()
    lr_scheduler_D_A.step()
    lr_scheduler_D_B.step()

    # Save models checkpoints
    torch.save(netG_A2B.state_dict(), 'output_EMG/netG_A2B.pth')
    torch.save(netG_B2A.state_dict(), 'output_EMG/netG_B2A.pth')
    torch.save(netD_A.state_dict(), 'output_EMG/netD_A.pth')
    torch.save(netD_B.state_dict(), 'output_EMG/netD_B.pth')

/home/filhoij/miniconda3/envs/research-dl1/lib/python3.11/site-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([5])) that is different to the input size (torch.Size([5, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch 001/125 [0090/0090] -- loss_G: 6.5592 | loss_G_identity: 1.7513 | loss_G_GAN: 1.2413 | loss_G_cycle: 3.5666 | loss_D: 0.2189 -- ETA: 2:59:54.42985321366810
Epoch 002/125 [0090/0090] -- loss_G: 5.5591 | loss_G_identity: 1.3289 | loss_G_GAN: 1.5385 | loss_G_cycle: 2.6918 | loss_D: 0.1211 -- ETA: 2:51:04.951032
Epoch 003/125 [0090/0090] -- loss_G: 5.4307 | loss_G_identity: 1.2200 | loss_G_GAN: 1.7280 | loss_G_cycle: 2.4826 | loss_D: 0.0909 -- ETA: 2:47:00.629367
Epoch 004/125 [0090/0090] -- loss_G: 5.2818 | loss_G_identity: 1.2244 | loss_G_GAN: 1.5620 | loss_G_cycle: 2.4954 | loss_D: 0.1441 -- ETA: 2:44:18.210136
Epoch 005/125 [0090/0090] -- loss_G: 5.5207 | loss_G_identity: 1.2217 | loss_G_GAN: 1.8136 | loss_G_cycle: 2.4854 | loss_D: 0.0488 -- ETA: 2:42:11.113684
Epoch 006/125 [0090/0090] -- loss_G: 5.3625 | loss_G_identity: 1.1676 | loss_G_GAN: 1.8130 | loss_G_cycle: 2.3819 | loss_D: 0.0600 -- ETA: 2:40:19.846602
Epoch 007/125 [0090/0090] -- loss_G: 5.2423 | loss_G_identity: 1.169

In [ ]:
torch.save(optimizer_G.state_dict(), 'output_EMG/optimizer_G.pth')
torch.save(optimizer_D_A.state_dict(), 'output_EMG/optimizer_D_A.pth')
torch.save(optimizer_D_B.state_dict(), 'output_EMG/optimizer_D_B.pth')

#learningrateschedulers

torch.save(lr_scheduler_G.state_dict(), 'output_EMG/lr_scheduler_G.pth')
torch.save(lr_scheduler_D_A.state_dict(), 'output_EMG/lr_scheduler_D_A.pth')
torch.save(lr_scheduler_D_B.state_dict(), 'output_EMG/lr_scheduler_D_B.pth')

In [ ]:
optimizer_G.load_state_dict(torch.load('output_EMG/optimizer_G.pth'))
optimizer_D_A.load_state_dict(torch.load('output_EMG/optimizer_D_A.pth'))
optimizer_D_B.load_state_dict(torch.load('output_EMG/optimizer_D_B.pth'))

#learningrateschedulers

lr_scheduler_G.load_state_dict(torch.load('output_EMG/lr_scheduler_G.pth'))
lr_scheduler_D_A.load_state_dict(torch.load('output_EMG/lr_scheduler_D_A.pth'))
lr_scheduler_D_B.load_state_dict(torch.load('output_EMG/lr_scheduler_D_B.pth'))